# SQL 

## Stillwater 

### Tiering analysis claims - Data pulls only





```{sql}

/* WRITING TO PERMANENT TABLES NOW */
use pricing
go

drop table if exists CLT.tiering_analysis_BPCLM_raw
go

/* aggregate claims dataset (at peril level) */
/* --> exclude ZP (original CP policies) (for both claim datasets) */
select * 
into CLT.tiering_analysis_BPCLM_raw
from openquery(DRORIONOLE,'select current date as data_pull_date, o.PLCNUM as Pol_Num, o.OCCMNR as Claim_Num, o.COMP# as Company, o.OCPLST as Pol_State, o.TMEFDT as Term_Eff_Date, o.ACCDAT as Acc_Date, o.REPDAT as Rep_Date, o.PERILCD as Peril_Code,
								sum(o2.@@PDA5) as MTD_Paid_Loss, sum(o2.@@SAA5) as MTD_Salv, sum(o2.@@SUA5) as MTD_Subr, sum(o2.@@ORA5) as MTD_Case_Res, sum(o2.@@PDB5) as MTD_ALAE_Paid, sum(o2.@@OEA5) as MTD_ALAE_Res, sum(o2.@@LIA5) as MTD_Inc_Loss, sum(o2.@@OLA5) as MTD_Orig_Loss_Res, sum(o2.@@CWA5) as MTD_CWP_Count, sum(o2.@@CNA5) as MTD_CWOP_Count, sum(o2.@@OPA5) as MTD_Rept_Count
							from ORION.DTA2PRD259.OPEU123 o -- claim occurrence
								left join ORION.DTA2PRD259.OPEU126 o2 -- claim financials
									on o.OCCMNR=o2.CECMNR
							group by o.OCCMNR, o.PLCNUM, o.COMP#, o.ACCDAT, o.REPDAT, o.TMEFDT, o.OCPLST, o.PERILCD
							having left(o.PLCNUM,2) in (''BC'',''BP'',''CM'',''CZ'',''XC'',''ZC'')
							order by o.PLCNUM, o.OCCMNR
						')

drop table if exists CLT.tiering_analysis_BPCLMTR_raw
go

drop table if exists #temp
go

/* transactional claims data */
select *
into #temp
from openquery(CLAIMSEDW, 'select pt.POLICY_NUMBER, pt.COMPANY, pt.STATE, case when left(c.CLAIM_NO,2)=''BP'' then right(c.CLAIM_NO,10) else c.CLAIM_NO end as CLAIM_NO, c.REPORTED_DATE, c.LOSS_DATE, ce.ISO_CODE, cc.COVERAGE_CODE, 
								  ct.ACTIVITY_TYPE, ct.ACTIVITY_CODE, ct.ALLOCATED, ct.PAID, ct.RECOVERED, ct.TRANSACTION_ID, ct.TRANSACTION_DATE,
								  trn.Case_Res, trn.ALAE_Res, trn.Paid_Loss, trn.ALAE_Paid, trn.ALAE_Rec, trn.Salv, trn.Subr, case when ct.ACTIVITY_TYPE = ''NEW'' then 1 else 0 end as Reported
						   from		(select CLAIM_ID, CLAIM_POLICY_TERM_ID, CLAIM_NO, REPORTED_DATE, LOSS_DATE, PCS_CAT_EVENT_ID, AO_CAT_EVENT_ID
									 from EDW.DBO.CLM_CLAIM
									 where LOB=''BP'' and CLAIM_IS_VALID_YN=''Y'' and DELETED_YN=''N'') c
								inner join 
									(select CLAIM_POLICY_TERM_ID, POLICY_NUMBER, COMPANY, STATE
									 from EDW.DBO.CLM_POL_TERM 
									 where TERM_IS_VALID_YN=''Y'' and DELETED_YN=''N'') pt
									on		pt.CLAIM_POLICY_TERM_ID = c.CLAIM_POLICY_TERM_ID 
								inner join 
									(select CLAIM_FEATURE_ID, CLAIM_ID, COVERAGE_CODE 
									 from EDW.DBO.CLM_FEATURE  
									 where FEATURE_IS_VALID_YN=''Y'' and DELETED_YN=''N'') cc
									on		c.CLAIM_ID = cc.CLAIM_ID
								inner join
									(select TRANSACTION_ID, CLAIM_FEATURE_ID, ACTIVITY_TYPE, ACTIVITY_CODE, ALLOCATED as ALLOCATED, PAID as PAID, RECOVERED as RECOVERED, 
											coalesce(SOURCE_MODIFIED_DATE, SOURCE_CREATED_DATE) as TRANSACTION_DATE
									 from EDW.DBO.CLM_TRANSACTION 
									 where TRANS_STATUS=''PROCESSED'' and DELETED_YN=''N'') ct
									on		cc.CLAIM_FEATURE_ID = ct.CLAIM_FEATURE_ID
								left join
									(select ISO_CODE, CAT_EVENT_ID
									from EDW.dbo.CLM_CAT_EVENT
									where cat_eve_is_valid_yn=''Y'') ce
									on		CAT_EVENT_ID = coalesce(c.PCS_CAT_EVENT_ID, c.AO_CAT_EVENT_ID)
								left join	(select sum(case when ACTIVITY_CODE=2001001 then ALLOCATED else 0 end) as Case_Res,
												sum(case when ACTIVITY_CODE=2001002 then ALLOCATED else 0 end) as ALAE_Res,
												sum(case when ACTIVITY_CODE=2002001 then PAID else 0 end) as Paid_Loss,
												sum(case when ACTIVITY_CODE=2002002 then PAID else 0 end) as ALAE_Paid,
												sum(case when ACTIVITY_CODE=2003002 then RECOVERED else 0 end) as ALAE_Rec,
												sum(case when ACTIVITY_CODE=2003003 then RECOVERED else 0 end) as Salv,
												sum(case when ACTIVITY_CODE=2003004 then RECOVERED else 0 end) as Subr,
												CLAIM_FEATURE_ID, TRANSACTION_ID, SOURCE_CREATED_DATE
											from EDW.dbo.CLM_TRANSACTION where deleted_yn=''N'' and TRANS_STATUS=''PROCESSED'' 
											group by CLAIM_FEATURE_ID, TRANSACTION_ID, SOURCE_CREATED_DATE) trn
										on TRN.TRANSACTION_ID=ct.TRANSACTION_ID
						where left(pt.POLICY_NUMBER,2) in (''BC'',''BP'',''CM'',''CZ'',''XC'',''ZC'')
						order by pt.POLICY_NUMBER, case when left(c.CLAIM_NO,2)=''BP'' then right(c.CLAIM_NO,10) else c.CLAIM_NO end
		')

select getdate() as data_pull_date, * 
into CLT.tiering_analysis_BPCLMTR_raw
from #temp
order by POLICY_NUMBER, CLAIM_NO

```